# Convolutional Net

## Importing Libraries and Data

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Useful Functions

In [2]:
def weights(shape, scale):
    return tf.Variable(tf.random_normal(shape, stddev = scale))

def bias(shape, value):
    return tf.Variable(tf.constant(value, shape = shape))

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxPooling(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

## Setting up the placeholders

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape =[None, 10])
keep_prob = tf.placeholder(tf.float32)

## Setting up the variables

In [4]:
W ={
    "1": weights([5,5,1,32], 0.1),
    "2": weights([5,5,32,64], 0.1),
    "FC": weights([7*7*64, 1024], 0.1),
    "Output": weights([1024, 10], 0.1)   
}


B = {
    "1": bias([32], 0.05),
    "2": bias([64], 0.05),
    "FC": bias([1024], 0.05),
    "Output":bias([10], 0.5)
}

## Defining the model

In [5]:
def model(x, W, B, keep_prob):
    
    x_image = tf.reshape(x, [-1,28,28,1])
    
    h_conv1 = tf.nn.relu(conv2d(x_image, W["1"]) + B["1"])
    h_pool1 = maxPooling(h_conv1)
    
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W["2"]) + B["2"])
    h_pool2 = maxPooling(h_conv2)
    h_pool2_flat = tf.reshape(h_pool2,[-1, 7*7*64])
    
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W["FC"]) + B["FC"])
    h_fc1_hidden = tf.nn.dropout(h_fc1, keep_prob)
    
    y_conv = tf.matmul(h_fc1_hidden, W["Output"]) + B["Output"]
    
    return y_conv

## Setting up Loss Function

In [6]:
y_conv = model(x,W,B,keep_prob)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_conv, labels = y))
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Stochastic Gradient Descent

In [7]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    for i in range(25000):
        batch = mnist.train.next_batch(111)

        if i%100 ==0:
            train_accuracy = sess.run(accuracy,feed_dict ={x: batch[0], y: batch[1], keep_prob : 1.0})
            print("Step %d, Training Accuracy %g"%(i, train_accuracy))
        
        _, xEntropy = sess.run([train_step, cross_entropy], feed_dict={x: batch[0], y: batch[1], keep_prob : 0.5})
        
    print("Test Accuracy %g"% sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob : 1.0}))

Step 0, Training Accuracy 0.117117
Step 100, Training Accuracy 0.954955
Step 200, Training Accuracy 0.954955
Step 300, Training Accuracy 0.954955
Step 400, Training Accuracy 0.972973
Step 500, Training Accuracy 0.954955
Step 600, Training Accuracy 0.990991
Step 700, Training Accuracy 0.981982
Step 800, Training Accuracy 0.981982
Step 900, Training Accuracy 1
Step 1000, Training Accuracy 1
Step 1100, Training Accuracy 0.990991
Step 1200, Training Accuracy 0.990991
Step 1300, Training Accuracy 1
Step 1400, Training Accuracy 0.981982
Step 1500, Training Accuracy 0.990991
Step 1600, Training Accuracy 0.990991
Step 1700, Training Accuracy 0.990991
Step 1800, Training Accuracy 0.990991
Step 1900, Training Accuracy 0.990991
Step 2000, Training Accuracy 0.990991
Step 2100, Training Accuracy 0.990991
Step 2200, Training Accuracy 0.990991
Step 2300, Training Accuracy 1
Step 2400, Training Accuracy 1
Step 2500, Training Accuracy 0.990991
Step 2600, Training Accuracy 1
Step 2700, Training Accuracy